## データセットを作成する

### [方針] 信州大学の電子情報システム工学科の教員の情報をスクレイピングで取得する。

In [1]:
# スクレイピング
import requests
from bs4 import BeautifulSoup
import re

# embeddingsのapi
import openai

# vector indexのapi
import pinecone

# api keyの管理
import os
from dotenv import load_dotenv

load_dotenv('.env')

/Users/yamamotoyuta/Library/Python/3.9/lib/python/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [2]:
# pineconeの設定
YOUR_API_KEY = os.getenv("YOUR_API_KEY")
YOUR_ENVIRONMENT = os.getenv("YOUR_ENVIRONMENT")

pinecone.init(api_key=YOUR_API_KEY, environment=YOUR_ENVIRONMENT)

In [254]:
# pineneのインデックスを生成する
# pinecone.create_index("shin-uni-teacher", dimension=1536, metric="euclidean")

In [255]:
# indexを削除する時に使用する

# pinecone.delete_index("shin-uni-teacher")

In [3]:
print(pinecone.list_indexes())
# indexを指定する
index = pinecone.Index("shin-uni-teacher")
# index の状況を確認
print(index.describe_index_stats())

['shin-uni-teacher']
{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 225}},
 'total_vector_count': 225}


In [4]:
openai.api_key = os.getenv("OPEN_AI_API_KEY")

def encode_text(input_message):
          
          response = openai.Embedding.create(
          input= input_message,
          model="text-embedding-ada-002"
          )
          embeddings = response['data'][0]['embedding']
          return embeddings

In [258]:
# teacher -> id
# id -> teacher
nametoid = {}
idtoname = {}

In [259]:
def web_scraping(teacher_id, pinecone_id, status, url, tag, class_name, in_tag = None, in_class_name = None) -> str:
    """ex: tag = h1, class_name = title"""
    # print(in_tag, in_class_name)
    response = requests.get(url)
    response.encoding = "utf-8"

    if response.status_code == 200: # response 成功 -> 200
        soup = BeautifulSoup(response.text, "html.parser")
        
        if in_tag is None and in_class_name is None:
            wrapper_div = soup.find(tag, class_=class_name)
        else:
            main_div = soup.find(in_tag, in_class_name)
            if tag == 'p':
                # p tag の時だけ何番目のpタグか指定する
                p_tags = main_div.find_all('p')
                # print(p_tags)
                wrapper_div = p_tags[class_name]
            else:
                wrapper_div = main_div.find(tag, class_=class_name)
        
        
        
        if wrapper_div:
            content = wrapper_div.text.strip()
            # print(content)
            
            # name -> id, id -> name
            if class_name == 'title01':
                print(content)
                nametoid[content] = teacher_id
                idtoname[teacher_id] = content # id < id+5を呼び出すことにする
                
            # openaiを呼ぶ
            vector = encode_text(content)

            # pinecodenのインデックス作成
            index.upsert(
                vectors=[{
                    'id': f'{pinecone_id}',
                    'values': vector,
                    'metadata': {
                              'teacher_id': teacher_id,
                              'name': idtoname[teacher_id],
                              'status': status,
                              'text': content,
                    }
            }]
            )
            
        else:
            print(f"{class_name}クラスが見つかりませんでした。")
    else:
        print("リクエストが失敗しました。")


In [260]:
# define tag name and class name

attribute_list = {}
# argument = (tag, class, None, None) if tag is in other tag, change None to other tag name. ex: div>.main>div>.clearfix
attribute_list['teacher_name'] = ('h1', 'title01')                    # 教員名
attribute_list['outline'] = ('h2', 'title02')                         # 研究の概要
attribute_list['discription'] = ('div', 'clearfix', 'div', 'main')    # 研究紹介
attribute_list['study-future'] = ('p', -3, 'div', 'main')             # 研究から始まる未来
attribute_list['graduation-future'] = ('p', -2, 'div', 'main')        # 卒業後の未来像
attribute_list['profile'] = ('p', -1, 'div', 'main')                  # プロフィール

In [274]:
# 教員名以外のリンク
pattern = r"cat" 

url = "https://www.shinshu-u.ac.jp/faculty/engineering/eict/introduction/teacher/"

response = requests.get(url)
response.encoding = "utf-8"

if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    wrapper_div = soup.find("div", class_="main")

    li_tags = wrapper_div.find_all("li")
    
    pinecone_id = 1
    teacher_id = 0
    for li in li_tags:
        
            # if teacher_id == 6:
            #     break
            
            link = li.find("a")
            if link:
                    href = link.get("href")
                    match = re.search(pattern, href)
                    if match is None:
                        # print(href)
                        teacher_id += 1
                        # test
                        if href != 'https://www.shinshu-u.ac.jp/faculty/engineering/eict/introduction/teacher/shibata.php':
                            for key_name in attribute_list.keys():
                                tmp = attribute_list[key_name]
                                web_scraping(teacher_id, pinecone_id, key_name, href, *tmp)
                                print(pinecone_id, teacher_id)
                                pinecone_id += 1
                                
                                
                                
                                
                        
else:
    print("リクエストが失敗しました。")


Asano, David Ken
1 1
2 1
3 1
4 1
5 1
6 1
伊東 栄次
7 2
8 2
9 2
10 2
11 2
12 2
Hernan, Aguirre
13 3
14 3
15 3
16 3
17 3
18 3
岡野 浩三
19 4
20 4
21 4
22 4
23 4
24 4
香山 瑞恵
25 5
26 5
27 5
28 5
29 5
30 5
Kawamoto, Pauline N.
31 6
32 6
33 6
34 6
35 6
36 6
小林 一樹
37 7
38 7
39 7
40 7
41 7
42 7
笹森 文仁
43 8
44 8
45 8
46 8
47 8
48 8
佐藤 敏郎
49 9
50 9
51 9
52 9
53 9
54 9
太子 敏則
55 10
56 10
57 10
58 10
59 10
60 10
田久 修
61 11
62 11
63 11
64 11
65 11
66 11
田中 清
67 12
68 12
69 12
70 12
71 12
72 12
橋本 佳男
73 13
74 13
75 13
76 13
77 13
78 13
藤原 洋志
79 14
80 14
81 14
82 14
83 14
84 14
丸山 稔
85 15
86 15
87 15
88 15
89 15
90 15
水野 勉
91 16
92 16
93 16
94 16
95 16
96 16
劉 小晰
97 17
98 17
99 17
100 17
101 17
102 17
和﨑 克己
103 18
104 18
105 18
106 18
107 18
108 18
阿部 誠
109 19
110 19
111 19
112 19
113 19
114 19
岡崎 裕之
115 20
116 20
117 20
118 20
119 20
120 20
小形 真平
121 21
122 21
123 21
124 21
125 21
126 21
川原 琢也
127 22
128 22
129 22
130 22
131 22
132 22
佐藤 真平
133 23
134 23
135 23
136 23
137 23
138 23
上口 光
139 24
140 24
141 24
14

## open aiのapiからembeddingsの値を得る

In [5]:
question = '進化計算の研究をしたいです'
text = encode_text(question)


In [6]:
result = index.query(
  vector=text,
  top_k=5,
  # include_values=True
  include_metadata=True
)

# print(len(result))
for match in result['matches']:
  print(match['metadata']['teacher_id'])
  print(match['metadata'])

  print(match['score'])
  # print(match["metadata"]['name'])


3.0
{'name': 'Hernan, Aguirre', 'status': 'outline', 'teacher_id': 3.0, 'text': '進化計算、多目的最適化、最適化の応用'}
0.179266691
12.0
{'name': '田中 清', 'status': 'outline', 'teacher_id': 12.0, 'text': '『画像処理』、『３次元データ処理』から『進化計算による多目的最適化』の研究'}
0.225562811
12.0
{'name': '田中 清', 'status': 'discription', 'teacher_id': 12.0, 'text': '進化計算による大規模・多数目的最適化問題解法のイメージ＝複雑化したシステムの設計・管理・運用における最適オプションを導き出す\n\nINRIAとの学術交流協定締結（右から二人目：INRIA総所長、左端：仏研究ｸﾞﾙｰﾌﾟ代表Marc Schoenauer博士、左から三人目：田中教授）\n\n田中研究室では、画像処理、画像符号化、画像分類など画像データに関連する技術、ドローン等で取得した３次元データの処理・解析技術、マルチメディアに対するデータハイディング技術、人間の視機能の解明、生物の遺伝と進化の過程を計算アルゴリズムとして具現した進化計算とこれを用いた多目的最適化技術、イノベーション創出のための実世界応用など、様々な情報処理技術の研究を行っています。海外の研究チームとの連携による国際共同研究、研究所や企業との共同研究を積極的に進めています。研究室には外国人研究者や留学生が多数在籍し、学術交流をグローバルに展開しています。'}
0.238800049
15.0
{'name': '丸山 稔', 'status': 'outline', 'teacher_id': 15.0, 'text': '機械学習に基づき、多様な情報を表現・理解する能力の獲得を目指して'}
0.287488222
7.0
{'name': '小林 一樹', 'status': 'study-future', 'teacher_id': 7.0, 'text': '人間の気持を理解し、その人に合わせて支援してくれる機械が普及すれば、効率や結果ばかりを重んじる現代社会が少しは良い方向

In [246]:
# teacher_id -> pinecone_id
# (6 * (teacher_id - 4)) + 4
def get_id(teacher_id):
          first = (6 * (teacher_id - 4)) + 4
          last = first + 4
          return [i for i in range(first, last+1)]

get_id(6)

[16, 17, 18, 19, 20]

In [290]:
result = [index.fetch(ids=[f'{i+1}']) for i in range(30)]

In [291]:
# skip = 9
# print(result)
for i in range(len(result)):
          # i += 1
          # if i == skip:
          #           skip += 6
          #           continue
          print(i, result[i]['vectors'][f'{i+1}']['metadata'])

0 {'name': 'Asano, David Ken', 'status': 'teacher_name', 'teacher_id': 1.0, 'text': 'Asano, David Ken'}
1 {'name': 'Asano, David Ken', 'status': 'outline', 'teacher_id': 1.0, 'text': '役に立つ情報通信・組込システム\u3000あなたも使いたくなる！'}
2 {'name': 'Asano, David Ken', 'status': 'discription', 'teacher_id': 1.0, 'text': 'Asano研究室では、幅広い範囲で情報システムの研究開発を行っています。『役に立つ』『使いやすい』を基本方針にし、人間の仕事や生活が『楽になる』『楽しくなる』システムを目指しています。多くのシステムは開発者の目線で作られていて、使用者が使いやすいものになっていません。システムが使いやすくなるためにどうしたらいいかを常に考えています。開発の例として、学習システム、センサーネットワークシステムがあります。'}
3 {'name': 'Asano, David Ken', 'status': 'study-future', 'teacher_id': 1.0, 'text': '複数の情報機器が『会話』をすれば、より高度な動作ができます。将来の情報機器は、単独ではなく、『仲間』と一緒に働いてくれるようになるでしょう。その具体的な方法や、それを実現するための技術を探るために、学生が日々研究や開発をしています。'}
4 {'name': 'Asano, David Ken', 'status': 'graduation-future', 'teacher_id': 1.0, 'text': '卒業生は、ソフトウェア業、電気メーカー、自動車メーカー等、様々な分野の企業に就職しています。学生には、研究のみならず、社会人として必要な知識や常識も学ばせています。'}
5 {'name': 'Asano, David Ken', 'status': 'profile', 'teacher_id': 1.0, 'text': '旧郵政省の通信総合研究所で客員研究員、信州大学講師・助教授・准教授

In [177]:
# index.delete(ids='5')

{}

In [218]:
idtoname[18.0]

'丸山 稔'

In [297]:
result = index.fetch(ids=['18']) 
result['vectors']['18']

{'id': '18',
 'metadata': {'name': 'Hernan, Aguirre',
              'status': 'profile',
              'teacher_id': 3.0,
              'text': 'Ecuadorian Engineer.\n'
                      'PhD from Shinshu University , 2003. Collaborates with '
                      'industry, promotes international exchange,\u3000and '
                      'conducts joint research with national and international '
                      'institutions.'},
 'values': [-0.0089125745,
            0.00180332258,
            0.00757395476,
            -0.00814963,
            -0.00310726347,
            0.0099945683,
            -0.0154808303,
            0.0106673464,
            -0.00326852221,
            -0.00703295786,
            -0.00337256,
            0.0212653335,
            0.041226726,
            0.00143832318,
            -0.0187268108,
            -0.0137538025,
            0.00793461874,
            -0.0183106605,
            0.00808720756,
            -0.00141578156,
            -0.0262